In [1]:
import h5py
import tangos as db
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import random

import foggie.utils.foggie_load as fog
import yt
from yt.units.yt_array import YTQuantity, YTArray
from yt.data_objects.particle_filters import add_particle_filter

import glob
from tangos.examples.misc import timestep_index

In [2]:
db.all_simulations()

[<Simulation("Tempest.9f11c.all.DD")>]

In [3]:
sim = db.get_simulation('Tempest.9f11c.all.DD')

In [4]:
all_sats = sim[-1][1].calculate('Satellites()')

sats = np.array([], dtype='object')
#starry_idx = np.array([21, 51, 109], dtype='object')
starry_idx = np.array([21], dtype='object')

for i in range(np.size(all_sats)):
    if all_sats[i].halo_number in starry_idx:
        sats = np.append(sats, all_sats[i]) # this ensures we're only checking for the verified starry halos
        
print(sats)

[<Halo 'Tempest.9f11c.all.DD/DD2427/DD2427/halo_21' | NDM=517 Nstar=0 Ngas=0>]


In [5]:
"""
First, we find all the properties of the progenitors 
for the star-forming surviving satellites.
"""

all_sat_cen = []
all_sat_rvir = []
all_sat_t = []

for sat_idx in starry_idx:
    if isinstance(sim[-1][sat_idx], type(None)) is False:
        mvir, cen, rvir, t = sim[-1][sat_idx].calculate_for_progenitors('Mvir', 'Center', 'Rvir', 't()')

        all_sat_cen.append(cen) # this compiles an array of all the progenitors' centers
        all_sat_rvir.append(rvir) # this compiles an array of all the progenitors' virial radii
        all_sat_t.append(t)

all_sat_cen = np.array(all_sat_cen[0], dtype='object')
all_sat_rvir = np.array(all_sat_rvir[0], dtype='object')
all_sat_t = np.array(all_sat_t[0], dtype='object')

print('Success!')

Success!


In [151]:
tslist = glob.glob('/Volumes/TempestTimeSteps/DD????/DD????') 
tslist.sort()

In [152]:
"""
Now, run through ALL the files with star data in the hard drive. This should take a while.
"""

halo = 'Tempest'
hnum = '008508'

halo_c_v_name = '/Users/bnguyen/FOGGIE/foggie/halo_infos/'+hnum+'/nref11c_nref9f/halo_c_v'
trackname = '/Users/bnguyen/FOGGIE/foggie/halo_tracks/'+hnum+'/nref11n_selfshield_15/halo_track_200kpc_nref9'
masses_dir = '/Users/bnguyen/FOGGIE/foggie/halo_infos/'+hnum+'/nref11c_nref9f/'

def StarParts(pfilter, data):
    return data[("all", "particle_type")] == 2 # only grab star particles

add_particle_filter("stars", function=StarParts, filtered_type='all', requires=["particle_type"])
#add_particle_filter("DM", function=DMParts, filtered_type='all', requires=["particle_type"])

def DMParts(pfilter, data):
    return data[("all", "particle_type")] == 4 # only grab DM particles

all_snaps = np.array([], dtype='object')
all_time_idx = np.array([], dtype='object')

for snap_name in tslist:
    ds, region = fog.foggie_load(snap_name, trackname, find_halo_center=True, halo_c_v_name=halo_c_v_name, disk_relative=False, \
                                particle_type_for_angmom='young_stars', do_filter_particles=False, gravity=False,\
                                region='refine_box',masses_dir=masses_dir)
    
    snapend = snap_name[-6:]
#    print(snapend)
    time_idx = timestep_index(sim, snapend)
    ads = ds.all_data()
    all_snaps = np.append(all_snaps, ds)
    all_time_idx = np.append(all_time_idx, time_idx)
    
print('SUCCESS!')

yt : [WARNING  ] 2022-07-19 18:01:10,579 The stars particle filter already exists. Overriding.


Opening snapshot /Volumes/TempestTimeSteps/DD0047/DD0047


yt : [INFO     ] 2022-07-19 18:01:10,819 Parameters: current_time              = 44.816527818285
yt : [INFO     ] 2022-07-19 18:01:10,822 Parameters: domain_dimensions         = [256 256 256]
yt : [INFO     ] 2022-07-19 18:01:10,828 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2022-07-19 18:01:10,834 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2022-07-19 18:01:10,835 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2022-07-19 18:01:10,836 Parameters: current_redshift          = 5.9180766004715
yt : [INFO     ] 2022-07-19 18:01:10,837 Parameters: omega_lambda              = 0.715
yt : [INFO     ] 2022-07-19 18:01:10,838 Parameters: omega_matter              = 0.285
yt : [INFO     ] 2022-07-19 18:01:10,838 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2022-07-19 18:01:10,839 Parameters: hubble_constant           = 0.695


get_refine_box: using this location:      col1       col2     col3     col4     col5     col6     col7   col8
------------- -------- -------- -------- -------- -------- -------- ----
5.91807660047 0.495372 0.494815 0.499965 0.497372 0.496815 0.501965    9


Parsing Hierarchy : 100%|████████████████| 2486/2486 [00:00<00:00, 10770.19it/s]
yt : [INFO     ] 2022-07-19 18:01:11,295 Gathering a field list (this may take a moment.)


...

Pulling halo center from catalog file
Opening snapshot /Volumes/TempestTimeSteps/DD2427/DD2427


yt : [INFO     ] 2022-07-19 18:11:56,784 Parameters: current_time              = 639.44480357087
yt : [INFO     ] 2022-07-19 18:11:56,787 Parameters: domain_dimensions         = [256 256 256]
yt : [INFO     ] 2022-07-19 18:11:56,790 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2022-07-19 18:11:56,793 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2022-07-19 18:11:56,794 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2022-07-19 18:11:56,794 Parameters: current_redshift          = 0
yt : [INFO     ] 2022-07-19 18:11:56,795 Parameters: omega_lambda              = 0.715
yt : [INFO     ] 2022-07-19 18:11:56,795 Parameters: omega_matter              = 0.285
yt : [INFO     ] 2022-07-19 18:11:56,796 Parameters: omega_radiation           = 0
yt : [INFO     ] 2022-07-19 18:11:56,797 Parameters: hubble_constant           = 0.695


get_refine_box: using this location:         col1          col2     col3     col4    col5     col6     col7  col8
------------------- -------- -------- ------- -------- -------- ------- ----
4.4408920985006e-16 0.488865 0.470316 0.50854 0.490865 0.472316 0.51054    9


Parsing Hierarchy : 100%|████████████████| 4262/4262 [00:00<00:00, 12984.24it/s]
yt : [INFO     ] 2022-07-19 18:11:57,376 Gathering a field list (this may take a moment.)


Pulling halo center from catalog file
SUCCESS!


In [155]:
"""
Now, we go through each of the satellites and get their respective mass profiles.
"""

bin_size = YTQuantity(0.25, 'kpc') # 250 pc is the resolution of the simulation

all_radius_arrays = []
starry_spheres = []

each_halo_radius_arrays = []    
each_halo_spheres = []
    
# this goes through the halo at each timestep to find its Rvir and center position
for idx in range(np.size(all_rvirs)):
    each_cen = YTArray(all_centers[idx], 'kpc')
    each_rvir = YTQuantity(all_rvirs[idx], 'kpc')
    ds = all_available_snaps[idx]
    
    ds.add_particle_filter('stars')
    ds.add_particle_filter('DM')
            
    # this uses yt to create spheres which contain the stars of each respective halo
    halo_sphere = ds.sphere(center=each_cen,radius=each_rvir)
    stars_loc = halo_sphere['stars','particle_position'].in_units('kpc')
    corrected_stars_loc = stars_loc - each_cen # this places the halo center as the origin of its stars' coords
    stars_mass = halo_sphere['stars','particle_mass'].in_units('Msun')
        
    if np.size(corrected_stars_loc) != 0:
        radius_array = np.arange(YTQuantity(0.01, 'kpc'), each_rvir, bin_size)

        title = "star_timesteps/star_location_" + str(tslist[idx][-6:] + '.txt')
        with open(title, "w") as f:
            for i in range(np.size(stars_mass)):
                f.write(str(corrected_stars_loc[i].value[0]) + ' ' + str(corrected_stars_loc[i].value[1]) + ' ' \
                    + str(corrected_stars_loc[i].value[2]) + ' ' + str(stars_mass[i].value) + ' ' \
                    + str(each_rvir.value) + ' ' + str(each_cen.value[0]) + ' ' + str(each_cen.value[1]) \
                    + ' ' + str(each_cen.value[2]) + '\n')

        each_halo_radius_arrays.append(radius_array)    
        each_halo_spheres.append(halo_sphere)

print('Success!')

Success!


In [160]:
#print(each_halo_radius_arrays)
print(each_halo_spheres)

[YTSphere (DD2007): , center=[1.83745762e+26 1.78307324e+26 1.91251607e+26] cm cm, radius=6.467031809757874e+22 cm, YTSphere (DD2027): , center=[1.85275127e+26 1.79725667e+26 1.92846172e+26] cm cm, radius=6.498289412865352e+22 cm, YTSphere (DD2047): , center=[1.86807796e+26 1.81145697e+26 1.94442424e+26] cm cm, radius=6.53383815900899e+22 cm, YTSphere (DD2067): , center=[1.88348095e+26 1.82572805e+26 1.96045311e+26] cm cm, radius=6.552010434866687e+22 cm, YTSphere (DD2087): , center=[1.89897426e+26 1.84000407e+26 1.97653611e+26] cm cm, radius=6.591507269956578e+22 cm, YTSphere (DD2107): , center=[1.91451817e+26 1.85433994e+26 1.99265161e+26] cm cm, radius=6.621524059951217e+22 cm, YTSphere (DD2127): , center=[1.93018018e+26 1.86871628e+26 2.00887224e+26] cm cm, radius=6.660917997683384e+22 cm, YTSphere (DD2147): , center=[1.94588619e+26 1.88312306e+26 2.02510386e+26] cm cm, radius=6.694977807162066e+22 cm, YTSphere (DD2167): , center=[1.96169770e+26 1.89760450e+26 2.04142783e+26] cm cm

In [154]:
"""
Now, we have to ensure that all the progenitors line up with the available 
timesteps in the star hard drive. To do that, we first have to convert 
all the time indices from the star hard drive to actual time in Gyr.
"""

available_timesteps = np.zeros(np.size(all_time_idx))
i = 0
while i < np.size(all_time_idx):
    available_timesteps[i] = sim[all_time_idx[i]].time_gyr
    i += 1
    
print('There are', np.size(available_timesteps), 'timesteps available in the star hard drive.')

There are 130 timesteps available in the star hard drive.


In [111]:
all_centers = []
all_rvirs = []
all_available_snaps = []

#for halo in range(np.size(starry_idx)):
each_halo_available_centers = []
each_halo_available_rvirs = []
each_halo_available_snaps = []
    
# Now, we compare the times in the snapshots to the available timesteps 
# in order to see how many snapshots are available in the star hard drive.
for time in available_timesteps:
    center = all_sat_cen[all_sat_t == time]
    rvir = all_sat_rvir[all_sat_t == time]
        
    if np.size(center) != 0 and np.size(rvir) != 0:
        center = YTArray(center[0] * 1000, 'kpc')
        rvir = YTQuantity(rvir[0], 'kpc')
            
        each_halo_available_centers.append(center)
        each_halo_available_rvirs.append(rvir)
        each_halo_available_snaps.append(all_snaps[available_timesteps == time][0])
            
each_halo_available_centers = np.array(each_halo_available_centers, dtype='object')
each_halo_available_rvirs = np.array(each_halo_available_rvirs, dtype='object')
each_halo_available_snaps = np.array(each_halo_available_snaps, dtype='object')
        
all_centers.append(each_halo_available_centers)
all_rvirs.append(each_halo_available_rvirs)
all_available_snaps.append(each_halo_available_snaps)
    
all_centers = np.array(all_centers[0], dtype='object')
all_rvirs = np.array(all_rvirs[0], dtype='object')
all_available_snaps = np.array(all_available_snaps[0], dtype='object')

print('success!')

success!
